In [3]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [7]:
'''
vote with three classifiers
'''

from sklearn import svm
import copy
import numpy as np
import pandas as pd
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
import os
import re
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=None
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录
        i=0
        for e in files:
            if re.search('predict.csv',e)!= None and re.search('human',e)!=None and re.search('xgboost',e)==None and re.search('XGBoost',e)==None:
                print i
                print e
                i=i+1
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[e[1] for index,e in enumerate(data) if data[index,0] == 1 ]#设置为probability:2或者predict_label:1
                negative_=[e[1] for index,e in enumerate(data) if data[index,0] == 0 ]#设置为probability:2或者predict_label:1
                positive_.extend(negative_)
                positive_=np.array(positive_).T
                positive_=positive_.reshape(1,-1)
                if X is None:
                    X=positive_
                else:
                    X=np.concatenate([X,positive_],axis=0)
    print np.array(X).shape
    print "the numbers of X",len(X)
    print "the length of X",len(X[0])
    numberCount=len(X)
    X=np.array(X).astype(int)
    X=X.reshape(numberCount,-1).T
    return X,np.array(Y)
X_,y_true=generating_integrated_training('data/',1130)
numberCount=len(X_[0])
cluster=numberCount
print cluster
I=0
J=0
K=0
ACC_=0
X_all=X_[:,0]
for i in xrange(1,cluster):
    X_all=X_all+X_[:,i]
data_=copy.deepcopy(X_all)
for ii,line_data in enumerate(data_):
    if data_[ii]>=1:
            data_[ii]=1
    else:
            data_[ii]=0
print y_true,data_
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, data_)
print "SN, SP",SN,SP

0
human_AthMethPre_GBDT_predict.csv
1
human_PC_PseDNC_test_SVM0.43[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
2
human_KNF_KNN_predict.csv
3
human_PC_PseDNC_test_SVM0.01[12]_predict.csv
4
human_binary_encoding_SVM_predict.csv
5
human_PC_PseDNC_test_SVM0.52[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
6
human_KNF_predict.csv
7
human_PSDSP_RF_predict.csv
8
human_knn_all_SVM_predict.csv
9
human_PC_PseDNC_test_SVM0.13[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
10
human_AthMethPre_KNN_predict.csv
11
human_PC_PseDNC_test_SVM0.65[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
12
human_knn2_LR_predict.csv
13
human_KSNPF_GBDT_predict.csv
14
human_PC_PseDNC_test_SVM0.34[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
15
human_knn2_NB_predict.csv
16
human_g-gap\:0_GBDT_predict.csv
17
human_knn_NB_predict.csv
18
human_PCP_SVM_predict.csv
19
human_PC_PseDNC_test_SVM0.91[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,

In [12]:
'''
vote with three classifiers and weighted voting
'''
from sklearn import svm
import copy
import numpy as np

import pandas as pd
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
import os
import re
import time
print "start"
startTime=time.time()
numberCount=100

def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=None
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录
        i=0
        for e in files:
            if re.search('predict.csv',e)!= None and re.search('human',e)!=None and re.search('xgboost',e)==None and re.search('XGBoost',e)==None:
                print i
                print e
                i=i+1
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[e[2] for index,e in enumerate(data) if data[index,0] == 1 ]#设置为probability:2或者predict_label:1
                negative_=[e[2] for index,e in enumerate(data) if data[index,0] == 0 ]#设置为probability:2或者predict_label:1

                positive_.extend(negative_)
                positive_=np.array(positive_).T
                positive_=positive_.reshape(1,-1)
                if X is None:
                    X=positive_
                else:
                    X=np.concatenate([X,positive_],axis=0)

    print "the numbers of X",len(X)
    print "the length of X",len(X[0])
    numberCount=len(X)
    X=np.array(X).astype(float)
    X=X.reshape(numberCount,-1).T
    return X,np.array(Y)
X_,y_true=generating_integrated_training('data/',1130)
numberCount=len(X_[0])
cluster=numberCount
I=0
J=0
K=0
L=0
M=0
N=0
ACC_=0
for i in xrange(cluster):
    for j in xrange(cluster):
        for k in xrange(cluster):
            final_data=X_[:,i]+X_[:,j]+X_[:,k]
            data_=copy.deepcopy(final_data)
            for ii,line_data in enumerate(data_):
                if data_[ii]*1.7>=0.5:
                        data_[ii]=1
                else:
                        data_[ii]=0
            data_=np.array(data_)
            data_=data_.astype(int)
            ACC=metrics.accuracy_score(y_true,data_)
            if ACC_<= ACC:
                ACC_=ACC
                I=i
                J=j
                K=k
                print ACC
                print I
                print J
                print K

endTime=time.time()
elapsedTime=startTime-endTime
print "end"
print "elaspsed time:",elapsedTime

start
0
human_AthMethPre_GBDT_predict.csv
1
human_PC_PseDNC_test_SVM0.43[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
2
human_KNF_KNN_predict.csv
3
human_PC_PseDNC_test_SVM0.01[12]_predict.csv
4
human_binary_encoding_SVM_predict.csv
5
human_PC_PseDNC_test_SVM0.52[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
6
human_KNF_predict.csv
7
human_PSDSP_RF_predict.csv
8
human_knn_all_SVM_predict.csv
9
human_PC_PseDNC_test_SVM0.13[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
10
human_AthMethPre_KNN_predict.csv
11
human_PC_PseDNC_test_SVM0.65[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
12
human_knn2_LR_predict.csv
13
human_KSNPF_GBDT_predict.csv
14
human_PC_PseDNC_test_SVM0.34[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
15
human_knn2_NB_predict.csv
16
human_g-gap\:0_GBDT_predict.csv
17
human_knn_NB_predict.csv
18
human_PCP_SVM_predict.csv
19
human_PC_PseDNC_test_SVM0.91[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1

In [ ]:
'''
vote with three classifiers and weighted voting
'''
from sklearn import svm
import copy
import numpy as np

import pandas as pd
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
import os
import re
import time
print "start"
startTime=time.time()
numberCount=100

def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=None
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录
        i=0
        for e in files:
            if re.search('predict.csv',e)!= None and re.search('human',e)!=None and re.search('xgboost',e)==None and re.search('XGBoost',e)==None:
                print i
                print e
                i=i+1
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[e[2] for index,e in enumerate(data) if data[index,0] == 1 ]#设置为probability:2或者predict_label:1
                negative_=[e[2] for index,e in enumerate(data) if data[index,0] == 0 ]#设置为probability:2或者predict_label:1

                positive_.extend(negative_)
                positive_=np.array(positive_).T
                positive_=positive_.reshape(1,-1)
                if X is None:
                    X=positive_
                else:
                    X=np.concatenate([X,positive_],axis=0)

    print "the numbers of X",len(X)
    print "the length of X",len(X[0])
    numberCount=len(X)
    X=np.array(X).astype(float)
    X=X.reshape(numberCount,-1).T
    return X,np.array(Y)
X_,y_true=generating_integrated_training('data/',1130)
numberCount=len(X_[0])
cluster=numberCount
I=0
J=0
K=0
L=0
M=0
N=0
ACC_=0
for i in xrange(cluster):
    for j in xrange(cluster):
        for k in xrange(cluster):
            for l in np.arange(1,2,0.1):
                for m in np.arange(1,2,0.1):
                    final_data=l*X_[:,i]+m*X_[:,j]+X_[:,k]
                    data_=copy.deepcopy(final_data)
                    for ii,line_data in enumerate(data_):
                        if data_[ii]>=0.5:
                                data_[ii]=1
                        else:
                                data_[ii]=0
                    data_=np.array(data_)
                    data_=data_.astype(int)
                    ACC=metrics.accuracy_score(y_true,data_)
                    if ACC_< ACC:
                        ACC_=ACC
                        I=i
                        J=j
                        K=k
                        L=l
                        M=m
                        print ACC
                        print I
                        print J
                        print K
                        print L
                        print M

endTime=time.time()
elapsedTime=startTime-endTime
print "end"
print "elaspsed time:",elapsedTime

start
0
human_AthMethPre_GBDT_predict.csv
1
human_PC_PseDNC_test_SVM0.43[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
2
human_PC_PseDNC_test_SVM0.01[1, 2, 3, 4, 5]_predict.csv
3
human_KNF_KNN_predict.csv
4
human_PC_PseDNC_test_SVM0.01[12]_predict.csv
5
human_binary_encoding_SVM_predict.csv
6
human_PC_PseDNC_test_SVM0.52[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
7
human_KNF_predict.csv
8
human_PSDSP_RF_predict.csv
9
human_knn_all_SVM_predict.csv
10
human_PC_PseDNC_test_SVM0.13[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
11
human_AthMethPre_KNN_predict.csv
12
human_PC_PseDNC_test_SVM0.65[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
13
human_knn2_LR_predict.csv
14
human_KSNPF_GBDT_predict.csv
15
human_PC_PseDNC_test_SVM0.34[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]_predict.csv
16
human_knn2_NB_predict.csv
17
human_g-gap\:0_GBDT_predict.csv
18
human_knn_NB_predict.csv
19
human_PCP_SVM_predict.csv
20
human_